# Multilayer Perceptron

Based on:

- http://gluon.mxnet.io/chapter03_deep-neural-networks/mlp-scratch.html

In [ ]:
require 'mxnet'

In [ ]:
@data_ctx = MXNet.cpu
@model_ctx = MXNet.cpu
#@model_ctx = MXNet.gpu

## Load Data

Download the data (unless it's already there):

In [ ]:
unless File.exist?('train-images-idx3-ubyte') &&
       File.exist?('train-labels-idx1-ubyte')
  system("wget http://data.mxnet.io/mxnet/data/mnist.zip")
  system("unzip -x mnist.zip")
end

In [ ]:
num_inputs = 784
num_outputs = 10
batch_size = 64
num_examples = 60000

train_iter = MXNet::IO::MNISTIter.new(
  batch_size: batch_size,
  shuffle: true)
test_iter = MXNet::IO::MNISTIter.new(
  image: 't10k-images-idx3-ubyte',
  label: 't10k-labels-idx1-ubyte',
  batch_size: batch_size,
  shuffle: false)
nil

## Set Parameters

Define the neural network's parameters:

In [ ]:
#######################
#  Set some constants so it's easy to modify the network later
#######################
num_hidden = 256
weight_scale = 0.01

#######################
#  Allocate parameters for the first hidden layer
#######################
@w1 = MXNet::NDArray.random_normal(shape: [num_inputs, num_hidden], scale: weight_scale, ctx: @model_ctx)
@b1 = MXNet::NDArray.random_normal(shape: [num_hidden], scale: weight_scale, ctx: @model_ctx)

#######################
#  Allocate parameters for the second hidden layer
#######################
@w2 = MXNet::NDArray.random_normal(shape: [num_hidden, num_hidden], scale: weight_scale, ctx: @model_ctx)
@b2 = MXNet::NDArray.random_normal(shape: [num_hidden], scale: weight_scale, ctx: @model_ctx)

#######################
#  Allocate parameters for the output layer
#######################
@w3 = MXNet::NDArray.random_normal(shape: [num_hidden, num_outputs], scale: weight_scale, ctx: @model_ctx)
@b3 = MXNet::NDArray.random_normal(shape: [num_outputs], scale: weight_scale, ctx: @model_ctx)

@params = [@w1, @b1, @w2, @b2, @w3, @b3]
nil

In [ ]:
@params.each do |param|
  param.attach_grad
end
nil

## Activation Function

In [ ]:
def relu(x)
  MXNet::NDArray.maximum(x, MXNet::NDArray.zeros_like(x))
end

## Softmax

In [ ]:
def softmax(y_linear)
  exp = MXNet::NDArray.exp(y_linear - MXNet::NDArray.max(y_linear))
  partition = MXNet::NDArray.nansum(exp, axis: 0, exclude: true).reshape([-1, 1])
  return exp / partition
end

## Cross Entropy Loss

In [ ]:
def cross_entropy(y_hat, y)
  return -MXNet::NDArray.nansum(y * MXNet::NDArray.log(y_hat), axis: 0, exclude: true)
end

In [ ]:
def softmax_cross_entropy(y_hat_linear, y)
  return -MXNet::NDArray.nansum(y * MXNet::NDArray.log_softmax(y_hat_linear), axis: 0, exclude: true)
end

## Model Definition

In [ ]:
def net(x)
  # first hidden layer
  h1_linear = MXNet::NDArray.dot(x, @w1) + @b1
  h1 = relu(h1_linear)

  # second hidden layer
  h2_linear = MXNet::NDArray.dot(h1, @w2) + @b2
  h2 = relu(h2_linear)

  # output layer
  y_hat_linear = MXNet::NDArray.dot(h2, @w3) + @b3
  return y_hat_linear
end

## Optimizer

In [ ]:
def sgd(params, lr)
  params.each do |param|
    param[0..-1] = param - lr * param.grad
  end
end

## Accuracy

In [ ]:
def evaluate_accuracy(data_iter)
  numerator = 0.0
  denominator = 0.0
  data_iter.each_with_index do |batch, i|
    data = batch.data[0].as_in_context(@model_ctx).reshape([-1, 784])
    label = batch.label[0].as_in_context(@model_ctx)
    output = net(data)
    predictions = MXNet::NDArray.argmax(output, axis: 1)
    numerator += MXNet::NDArray.sum(predictions == label)
    denominator += data.shape[0]
  end
  return (numerator / denominator).as_scalar
end

In [ ]:
epochs = 10
learning_rate = 0.001
smoothing_constant = 0.01

epochs.times do |e|
  start = Time.now
  cumulative_loss = 0.0
  train_iter.each_with_index do |batch, i|
    data = batch.data[0].as_in_context(@model_ctx).reshape([-1, 784])
    label = batch.label[0].as_in_context(@model_ctx)
    label_one_hot = MXNet::NDArray.one_hot(label, depth: 10)
    loss = MXNet::Autograd.record do
      output = net(data)
      softmax_cross_entropy(output, label_one_hot)
    end
    loss.backward()
    sgd(@params, learning_rate)
    cumulative_loss += MXNet::NDArray.sum(loss).as_scalar
  end
  
  test_accuracy = evaluate_accuracy(test_iter)
  train_accuracy = evaluate_accuracy(train_iter)
  duration = Time.now - start
  puts "Epoch #{e}. Loss: #{cumulative_loss/num_examples}, Train_acc #{train_accuracy}, Test_acc #{test_accuracy} (#{duration} sec)"
end

## Classification

In [ ]:
require 'chunky_png'
require 'base64'

def imshow(ary)
  height, width = ary.shape
  fig = ChunkyPNG::Image.new(width, height, ChunkyPNG::Color::TRANSPARENT)
  ary = ((ary - ary.min) / ary.max) * 255
  0.upto(height - 1) do |i|
    0.upto(width - 1) do |j|
      v = ary[i, j].round
      fig[j, i] = ChunkyPNG::Color.rgba(v, v, v, 255)
    end
  end

  src = 'data:image/png;base64,' + Base64.strict_encode64(fig.to_blob)
  IRuby.display "<img src='#{src}' width='#{width*2}' height='#{height*2}' />", mime: 'text/html'
end

In [ ]:
# Define the funtion to do prediction
def model_predict(data)
  output = net(data)
  MXNet::NDArray.argmax(output, axis: 1)
end

samples = 10
sample_iter = test_iter = MXNet::IO::MNISTIter.new(
  image: 't10k-images-idx3-ubyte',
  label: 't10k-labels-idx1-ubyte',
  batch_size: samples,
  shuffle: true)
sample_iter.each do |batch|
  batch = sample_iter.next_batch
  data = batch.data[0].as_in_context(@model_ctx)
  label = batch.label[0]

  im = data.transpose(axes: [1, 0, 2, 3]).reshape([10*28, 28, 1])
  imshow(im[0..-1, 0..-1, 0].to_narray)

  pred = model_predict(data.reshape([-1, 784]))
  puts "model predictions are: #{pred.inspect}"
  puts
  puts "true labels: #{label.inspect}"
  break
end